In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)

#alunos_df = pd.read_csv("C:/Users/pedro/Downloads/TRI/microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv", nrows= 6 * 1000 * 1000, sep=";",  encoding='latin')

columns_to_read = ['TP_LINGUA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_MT', 'TX_RESPOSTAS_LC', 'TX_GABARITO_CN', 'TX_GABARITO_MT', 'TX_GABARITO_CH', 'TX_GABARITO_LC']

year = 2022
alunos_df = pd.read_csv(f"../../data/raw-enem-exams/microdados_enem_{year}/DADOS/MICRODADOS_ENEM_{year}.csv", usecols=columns_to_read, nrows= 6 * 1000 * 1000, sep=";",  encoding='latin')
item_df_path = f"../../data/raw-enem-exams/microdados_enem_{year}/DADOS/ITENS_PROVA_{year}.csv"

alunos_df.dropna(subset=['TX_GABARITO_CH', 'TX_GABARITO_MT', 'TX_GABARITO_CN', 'TX_GABARITO_LC'], inplace=True)

alunos_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2344823 entries, 2 to 3476102
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   CO_PROVA_CN      float64
 1   CO_PROVA_CH      float64
 2   CO_PROVA_LC      float64
 3   CO_PROVA_MT      float64
 4   NU_NOTA_CN       float64
 5   NU_NOTA_CH       float64
 6   NU_NOTA_LC       float64
 7   NU_NOTA_MT       float64
 8   TX_RESPOSTAS_CN  object 
 9   TX_RESPOSTAS_CH  object 
 10  TX_RESPOSTAS_LC  object 
 11  TX_RESPOSTAS_MT  object 
 12  TP_LINGUA        int64  
 13  TX_GABARITO_CN   object 
 14  TX_GABARITO_CH   object 
 15  TX_GABARITO_LC   object 
 16  TX_GABARITO_MT   object 
dtypes: float64(8), int64(1), object(8)
memory usage: 322.0+ MB


In [2]:
def grade_CTT(row, tipo_prova):
  
    student_responses =  [char for char in row['TX_RESPOSTAS_' + tipo_prova]] 
    correct_responses = [char for char in row['TX_GABARITO_' + tipo_prova]]

    # Verifique se os vetores têm o mesmo comprimento
    if tipo_prova == 'LC' and len(correct_responses) == 50:
        # Skip Spanish exam.
         correct_responses = correct_responses[5:]
    if tipo_prova == 'LC' and len(student_responses) == 50:
        # Skip Spanish exam.
         student_responses = student_responses[5:]
        
    if len(student_responses) != len(correct_responses):
        return "Os vetores têm comprimentos diferentes"
    
    # Inicialize a contagem de respostas corretas
    correct_count = 0
    
    # Percorra os vetores e conte as respostas corretas
    for i in range(len(student_responses)):
        if student_responses[i] == correct_responses[i]:
            correct_count += 1
    
    return correct_count

# Exemplo de uso da função:
#student_answers = "AAAABCCC"
#correct_answers = "AAAABBBB"

#num_correct = grade_CTT(student_answers, correct_answers)
#print("Número de respostas corretas:", num_correct)

In [3]:
alunos_df['CO_PROVA_CH'].value_counts()

CO_PROVA_CH
1057.0    584266
1055.0    576449
1058.0    576412
1056.0    575951
1178.0      7479
1177.0      7400
1176.0      7346
1175.0      7332
1063.0       928
1062.0       335
1136.0       242
1135.0       239
1137.0       233
1138.0       211
Name: count, dtype: int64

In [4]:
def response_pattern(row, tipo_prova):

    response_pattern = ""
    student_responses =  [char for char in row['TX_RESPOSTAS_' + tipo_prova]] 
    correct_responses = [char for char in row['TX_GABARITO_' + tipo_prova]]
   
    # Verifique se os vetores têm o mesmo comprimento
    if tipo_prova == 'LC' and len(correct_responses) == 50:
        # Skip Spanish exam.
         correct_responses = correct_responses[5:]
    if tipo_prova == 'LC' and len(student_responses) == 50:
        # Skip Spanish exam.
         student_responses = student_responses[5:]
        
    if len(student_responses) != len(correct_responses):
        raise Exception(f"Os vetores têm comprimentos diferentes. Estudante respondeu {len(student_responses)} e o gabarito tem {len(correct_responses)}")
    
    # Inicialize a contagem de respostas corretas
    correct_count = 0
    
    # Percorra os vetores e conte as respostas corretas
    for i in range(len(student_responses)):
        if student_responses[i] == correct_responses[i]:
            response_pattern += "1"
        else:       
            response_pattern += "0"
    
    return response_pattern

# Exemplo de uso da função:
#student_answers = "AAAABCCC"
#correct_answers = "AAAABBBB"

#num_correct = grade_CTT(student_answers, correct_answers)
#print("Número de respostas corretas:", num_correct)

In [5]:
def tx_gabarito(row, tipo_prova):

    response_pattern = ""
    student_responses =  [char for char in row['TX_RESPOSTAS_' + tipo_prova]] 
    correct_responses = [char for char in row['TX_GABARITO_' + tipo_prova]]
   
    # Verifique se os vetores têm o mesmo comprimento
    if tipo_prova == 'LC' and len(correct_responses) == 50:
        # Skip Spanish exam.
         correct_responses = correct_responses[5:]
    if tipo_prova == 'LC' and len(student_responses) == 50:
        # Skip Spanish exam.
         student_responses = student_responses[5:]
        
    if len(student_responses) != len(correct_responses):
        raise Exception(f"Os vetores têm comprimentos diferentes. Estudante respondeu {len(student_responses)} e o gabarito tem {len(correct_responses)}")
    
    # Inicialize a contagem de respostas corretas
    correct_count = 0
    
    # Percorra os vetores e conte as respostas corretas
    for i in range(len(student_responses)):
        if student_responses[i] == correct_responses[i]:
            response_pattern += "1"
        else:       
            response_pattern += "0"
    
    return ''.join(correct_responses)


def tx_respostas(row, tipo_prova):

    response_pattern = ""
    student_responses =  [char for char in row['TX_RESPOSTAS_' + tipo_prova]] 
    correct_responses = [char for char in row['TX_GABARITO_' + tipo_prova]]
   
    # Verifique se os vetores têm o mesmo comprimento
    if tipo_prova == 'LC' and len(correct_responses) == 50:
        # Skip Spanish exam.
         correct_responses = correct_responses[5:]
    if tipo_prova == 'LC' and len(student_responses) == 50:
        # Skip Spanish exam.
         student_responses = student_responses[5:]
        
    if len(student_responses) != len(correct_responses):
        raise Exception(f"Os vetores têm comprimentos diferentes. Estudante respondeu {len(student_responses)} e o gabarito tem {len(correct_responses)}")
    
    # Inicialize a contagem de respostas corretas
    correct_count = 0
    
    # Percorra os vetores e conte as respostas corretas
    for i in range(len(student_responses)):
        if student_responses[i] == correct_responses[i]:
            response_pattern += "1"
        else:       
            response_pattern += "0"
    
    return ''.join(student_responses)


In [6]:
print("Computing CTTs...")

'''
alunos_df['CTT_SCORE_CH'] = alunos_df.apply(grade_CTT, axis=1,  args=('CH',))
alunos_df['RESPONSE_PATTERN_CH'] = alunos_df.apply(response_pattern, axis=1, args=('CH',))

alunos_df['CTT_SCORE_MT'] = alunos_df.apply(grade_CTT, axis=1,  args=('MT',))
alunos_df['RESPONSE_PATTERN_MT'] = alunos_df.apply(response_pattern, axis=1, args=('MT',))

alunos_df['CTT_SCORE_CN'] = alunos_df.apply(grade_CTT, axis=1,  args=('CN',))
alunos_df['RESPONSE_PATTERN_CN'] = alunos_df.apply(response_pattern, axis=1, args=('CN',))
'''

alunos_df['CTT_SCORE_LC'] = alunos_df.apply(grade_CTT, axis=1,  args=('LC',))
alunos_df['RESPONSE_PATTERN_LC'] = alunos_df.apply(response_pattern, axis=1, args=('LC',))
alunos_df['TX_GABARITO_LC'] = alunos_df.apply(tx_gabarito, axis=1, args=('LC',))
alunos_df['TX_RESPOSTAS_LC'] = alunos_df.apply(tx_respostas, axis=1, args=('LC',))

print("CTTs computed!")

Computing CTTs...
CTTs computed!


In [7]:
alunos_df['TP_LINGUA'].unique()

array([1, 0])

In [10]:
# MT
'''
alunos_df[['TX_RESPOSTAS_MT','TX_GABARITO_MT', 'NU_NOTA_MT', 'CTT_SCORE_MT', 'CO_PROVA_MT', 'RESPONSE_PATTERN_MT']] \
 .sort_values(by='NU_NOTA_MT', ascending=False) \
 .rename(columns={'TX_RESPOSTAS_MT': 'TX_RESPOSTAS', 'TX_GABARITO_MT': 'TX_GABARITO', 'NU_NOTA_MT': 'NU_NOTA', 'CTT_SCORE_MT': 'CTT_SCORE', 'CO_PROVA_MT': 'CO_PROVA',  'RESPONSE_PATTERN_MT': 'RESPONSE_PATTERN'}) \
 .to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/test_responses_humans_MT_{year}.csv")

# CH
alunos_df[['TX_RESPOSTAS_CH','TX_GABARITO_CH', 'NU_NOTA_CH', 'CTT_SCORE_CH', 'CO_PROVA_CH', 'RESPONSE_PATTERN_CH']] \
 .sort_values(by='NU_NOTA_CH', ascending=False) \
 .rename(columns={'TX_RESPOSTAS_CH': 'TX_RESPOSTAS', 'TX_GABARITO_CH': 'TX_GABARITO', 'NU_NOTA_CH': 'NU_NOTA', 'CTT_SCORE_CH': 'CTT_SCORE', 'CO_PROVA_CH': 'CO_PROVA',  'RESPONSE_PATTERN_CH': 'RESPONSE_PATTERN'}) \
 .to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/test_responses_humans_CH_{year}.csv")

# CN
alunos_df[['TX_RESPOSTAS_CN','TX_GABARITO_CN', 'NU_NOTA_CN', 'CTT_SCORE_CN', 'CO_PROVA_CN', 'RESPONSE_PATTERN_CN']] \
 .sort_values(by='NU_NOTA_CN', ascending=False) \
 .rename(columns={'TX_RESPOSTAS_CN': 'TX_RESPOSTAS', 'TX_GABARITO_CN': 'TX_GABARITO', 'NU_NOTA_CN': 'NU_NOTA', 'CTT_SCORE_CN': 'CTT_SCORE', 'CO_PROVA_CN': 'CO_PROVA',  'RESPONSE_PATTERN_CN': 'RESPONSE_PATTERN'}) \
 .to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/test_responses_humans_CN_{year}.csv")
'''

# LC
# Keep only studes who chose the Spanish exam instead of English.
alunos_df[alunos_df['TP_LINGUA'] == 0][['TX_RESPOSTAS_LC','TX_GABARITO_LC', 'NU_NOTA_LC', 'CTT_SCORE_LC', 'CO_PROVA_LC', 'RESPONSE_PATTERN_LC', 'TP_LINGUA']] \
 .sort_values(by='NU_NOTA_LC', ascending=False) \
 .rename(columns={'TX_RESPOSTAS_LC': 'TX_RESPOSTAS', 'TX_GABARITO_LC': 'TX_GABARITO', 'NU_NOTA_LC': 'NU_NOTA', 'CTT_SCORE_LC': 'CTT_SCORE', 'CO_PROVA_LC': 'CO_PROVA',  'RESPONSE_PATTERN_LC': 'RESPONSE_PATTERN'}) \
 .to_csv(f"../../data/enem-human-responses/test_responses_humans_LC_{year}.csv")

alunos_df[alunos_df['TP_LINGUA'] == 0].head()

,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,CTT_SCORE_LC,RESPONSE_PATTERN_LC
17,1086.0,1055.0,1065.0,1076.0,396.8,528.7,551.4,536.0,ABEACDCACEADBDBDBAEBCDCABBBEDCBEBDACDAAECEDAE,CEABADCEDEBAECBADABBCDBDAAEDCEDBABCACBEDAEDEA,CCBDBECEAECBDDECACCDBACECCBADBCAADACCCDDBCEDE,CABACCAEEAABEDCCCBBECCBAEBEBCCDBDBAEDEDABEAEB,0,EACDDEBEACCCDABCEECBBCCAEBEDABEADBADDCBDDAAEB,DEABEEDAEBAAECBECBABCDDAAADBCEDAEACDBBCDCBECA,EDCAAEAAACABBDBEBACCBEEECCBBDCAAADCCABEDBDECE,DBAAACEBEDAECCBECDEECBDCABECBEABDDCCDBDXCBAAC,19,000001001001010000101001111010011101000110101
20,1086.0,1055.0,1065.0,1076.0,466.5,476.2,488.3,462.7,CCDDCEEABDDDCABEDBEABCCBBCCEEDDEBCBCBDEBEECDB,BEADADABEEABDBBDBEABCDCBBEEECCABDBEACDDDAEDCE,CCDDBEEDAEDBCDEBECCDDAEDBCEDCAABEDDCECADBACAE,CBDCABEDEBABBEDBAEABCDDBDEBAAECBEDDBBCADCCECA,0,EACDDEBEACCCDABCEECBBCCAEBEDABEADBADDCBDDAAEB,DEABEEDAEBAAECBECBABCDDAAADBCEDAEACDBBCDCBECA,EDCAAEAAACABBDBEBACCBEEECCBBDCAAADCCABEDBDECE,DBAAACEBEDAECCBECDEECBDCABECBEABDDCCDBDXCBAAC,13,000001001001010000100010010000100101000110001
25,1086.0,1055.0,1065.0,1076.0,386.5,458.5,479.7,383.1,DEBCDCEBDEBACDCBADBCECBEBDECADBD*BCDAEDCEDBAD,CAEBEDDAEDADBCDBDAABDBEAEDACCBACBDCEBDEDCACDE,BCEBEBDEBAACDBEDEACDEAEEBECBDEBEADACACBCBDECD,ABDCDABDCDECEBACECDCBAEBCADBCEADDBECBACEBAECE,0,EACDDEBEACCCDABCEECBBCCAEBEDABEADBADDCBDDAAEB,DEABEEDAEBAAECBECBABCDDAAADBCEDAEACDBBCDCBECA,EDCAAEAAACABBDBEBACCBEEECCBBDCAAADCCABEDBDECE,DBAAACEBEDAECCBECDEECBDCABECBEABDDCCDBDXCBAAC,15,000000000010000001100011000110001101100011110
27,1086.0,1055.0,1065.0,1076.0,414.6,518.2,596.5,573.4,EBCDAACABEABDCABDAECBADACDBAEDCEBBDEAEDAAEBDD,DCACEECAEBADECAACBBBBADCECDEACCADBBEBDABBEDCA,DCBDEEAEAAACCDBCBECDEEEECCBBDCEAEEBCADADBAEDE,AACCECBEEDDCDEBDADCCBEEBDCABBDABDCCEAEDBAEBCD,0,EACDDEBEACCCDABCEECBBCCAEBEDABEADBADDCBDDAAEB,DEABEEDAEBAAECBECBABCDDAAADBCEDAEACDBBCDCBECA,EDCAAEAAACABBDBEBACCBEEECCBBDCAAADCCABEDBDECE,DBAAACEBEDAECCBECDEECBDCABECBEABDDCCDBDXCBAAC,24,000001101010011010100111111111010001100110101
29,1087.0,1056.0,1066.0,1078.0,437.0,505.2,438.6,615.3,AEBDECABEBBCCBDBBBBECDCCBEAEAEAEBADBBCDCCEDBC,DDEABCEEDBEAEEECEACEADECACEDADACDBAACAEACAADA,DEDEDDDDBDCADECEADDBEEDBBCEEBEEDDEBEEBCBBECEB,BBEDABEBDACDEBBCECEABECBCBCABCDCDBCBCDBCBBDAA,0,DDECDBEACCAEBEAEBBCCDDCBDDACBEACEABCEABEDADBA,ECBABCDDAAECBCBECAEBAEACDEEDABBCDDEABCEDAAADB,DCEAADDBCABEDCAAADCCECEEBABEEAAABDBCCCABEECBB,BEEDAEABDDCEBDBAAAAACXCBCCCBCCDBDEECBDCABEECD,14,100001100000100011001000100100000010000101101


In [11]:
alunos_df.iloc[0, :]

CO_PROVA_CN                                                   1087.0
CO_PROVA_CH                                                   1056.0
CO_PROVA_LC                                                   1066.0
CO_PROVA_MT                                                   1078.0
NU_NOTA_CN                                                     421.1
NU_NOTA_CH                                                     546.0
NU_NOTA_LC                                                     498.8
NU_NOTA_MT                                                     565.3
TX_RESPOSTAS_CN        DCDCEBADDABEDBCBEAEACCDDECEEBECBECAAECDDDDBCE
TX_RESPOSTAS_CH        DBDABCADADECACBDCCEDACCCECDDABDCEADABCBCBAEAB
TX_RESPOSTAS_LC        ABEABADCCADCAAAACDADDCECBAAACADABDEBCBCAAACEB
TX_RESPOSTAS_MT        BDEBACECBCAEBABEDBABDACBCABCDEDEADBAABCBEACCD
TP_LINGUA                                                          1
TX_GABARITO_CN         DDECDBEACCAEBEAEBBCCDDCBDDACBEACEABCEABEDADBA
TX_GABARITO_CH         ECBABCDDAAE

In [1]:
import pandas as pd
from scipy.stats import entropy
from collections import defaultdict
from scipy.stats import percentileofscore

def generate_human_histogram_df(responses_df, PROVA, CO_PROVA, o, min_nota, max_nota):

    #responses_df = responses_df[responses_df['CO_PROVA_'+PROVA] == CO_PROVA]
   
    itens_df = pd.read_csv(item_df_path, sep=';', encoding = "latin")
    itens_df = itens_df[itens_df['SG_AREA'] == PROVA]
    itens_df = itens_df[itens_df['TP_LINGUA'] != 0]
    print('len_itens_df', len(itens_df))
    
    itens_df = itens_df.sort_values(by=['CO_PROVA','CO_POSICAO'], ascending=True) 
    
    map_co_item_to_target_item_id = dict()
    target_item_id = 0
    for idx, item in itens_df.iterrows():
        co_item = item['CO_ITEM']
        co_posicao = item['CO_POSICAO']
        co_prova = item['CO_PROVA']
        gabarito = item['TX_GABARITO']

        if co_prova == CO_PROVA:
            target_item_id += 1
            map_co_item_to_target_item_id[co_item] = target_item_id


        #if co_item == 111664 and co_prova:
        #    print('co_item', co_item, 'co_posicao', co_posicao, 'co_prova', co_prova, 'gabarito', gabarito, 'target_item_id', target_item_id)
    
    #print('map_co_item_to_target_item_id', map_co_item_to_target_item_id)
    
    map_co_prova_co_item_to_target_prova_item_id = defaultdict(dict)
    map_co_prova_item_id_to_co_item = defaultdict(dict)
    dic_item_id_count = defaultdict(int)
    for idx, item in itens_df.iterrows():
        co_item = item['CO_ITEM']
        co_posicao = item['CO_POSICAO']
        co_prova = item['CO_PROVA']
        gabarito = item['TX_GABARITO']

        dic_item_id_count[co_prova] += 1

        if co_item not in map_co_item_to_target_item_id:
            #print('co_item', co_item, 'not found in map_co_item_to_target_item_id')
            continue

        #print(co_prova, 'co_posicao', co_posicao, 'item_id', dic_item_id_count[co_prova], gabarito)
        
        #if co_item == 111664:
        #    print('co_item', co_item, 'co_posicao', co_posicao, 'co_prova', co_prova, 'gabarito', gabarito, 'dic_item_id_count[co_prova]', dic_item_id_count[co_prova])
        
        map_co_prova_item_id_to_co_item[co_prova][dic_item_id_count[co_prova]]=co_item
        map_co_prova_co_item_to_target_prova_item_id[co_prova][co_item] = map_co_item_to_target_item_id[co_item]

        #print('co_prova', co_prova, 'co_posicao', co_posicao, 'item', dic_item_id_count[co_prova], 'is',co_item)

    #print('map_co_prova_item_id_to_co_item contains PROVAS', map_co_prova_item_id_to_co_item.keys())
    #for co_prova in map_co_prova_item_id_to_co_item.keys():
        #print('prova ', co_prova, 'contains itens', map_co_prova_item_id_to_co_item[co_prova].keys())

    # ACERTA ITENS_DF
    itens_df = itens_df[itens_df['CO_PROVA'] == CO_PROVA].sort_values(by='CO_POSICAO', ascending=True)
    itens_df.reset_index()
    itens_df['human_item_id'] = range(1, len(itens_df) + 1)
    
    #print('len_responses_df', len(responses_df))
    #print('map_co_prova_item_id_to_co_item', map_co_prova_item_id_to_co_item)
    
    options =  ['A', 'B', 'C', 'D', 'E']
    
    histograms_options = {}
    histograms_correct = { }
    correct_options = list()
    correct_response_pattern = ""

    student_count = 0

    record_count = 0

    filtered_responses_df = responses_df[(responses_df['NU_NOTA'] >= min_nota) & (responses_df['NU_NOTA'] <= max_nota)]
    
    dic_correct_option = dict()
    for index, student in filtered_responses_df.iterrows():      
        #if student['TP_PRESENCA_' + PROVA] != 1:
        #    continue  
    
        co_prova = student['CO_PROVA']

        if co_prova not in dic_item_id_count:
            continue
        
        #if co_prova != CO_PROVA:
        #    continue
        
        student_count += 1
        #if student_count % 10000 == 0:
       #     print('student_count', student_count)

        #if student_count == 200000:
        #    break

        record_count += 1

        for item_id, choice_original_order in enumerate(student['TX_RESPOSTAS'], start=1):
            
            #if co_prova != CO_PROVA:
            #    continue
            
            #shuffled_response_pattern = item['SHUFFLED_ORDER_TX_RESPOSTAS']
            response_pattern = student['TX_RESPOSTAS']
            #shuffled_correct_response_pattern = item['SHUFFLED_ORDER_TX_GABARITO']
            correct_response_pattern = student['TX_GABARITO']
        
            #choice_shuffled_order = shuffled_response_pattern[item_id-1]
            #correct_shuffled_order = shuffled_correct_response_pattern[item_id-1]

            choice_original_order = response_pattern[item_id-1]
            correct_original_order = correct_response_pattern[item_id-1]

            if item_id not in map_co_prova_item_id_to_co_item[co_prova]:
                #print('item_id', item_id, 'not in co_prova', co_prova)
                continue
                
            co_item = map_co_prova_item_id_to_co_item[co_prova][item_id]
            target_item_id = map_co_prova_co_item_to_target_prova_item_id[co_prova][co_item]
            correct = choice_original_order == correct_original_order

            #if co_item == 111664:
            #    print("Adding for co_prova", co_prova, "co_item", co_item, "item_id", item_id, "target_item_id", target_item_id, "correct", correct_original_order)
            #    print("AAAH! correct option for target_item_id " + str(target_item_id) + " differs. " \
            #                    "item_id is",item_id, \
            #                    "co_prova is",co_prova,  \
            #                    "gabarito is", correct_response_pattern)
            
            if target_item_id in dic_correct_option and dic_correct_option[target_item_id] != correct_original_order:
                raise Exception("Fatal error: correct option for target_item_id " + str(target_item_id) + " differs. " \
                                "item_id is",item_id,
                                "co_prova is",co_prova,  \
                                "gabarito is", correct_response_pattern, \
                                "It is " + dic_correct_option[target_item_id] + "but we're trying to change to " + correct_original_order)

            dic_correct_option[target_item_id] = correct_original_order
            
            # Update the histogram for the current position and letter
            if target_item_id not in histograms_options:
                histograms_options[target_item_id] = { }
                for option in options:
                    histograms_options[target_item_id][option] = 0
                histograms_correct[target_item_id] = { }

                        
            if o == 'choice_original_order':
                letter = choice_original_order
                if letter not in options:
                    continue
                histograms_options[target_item_id][letter] = histograms_options[target_item_id].get(letter, 0) + 1
                          
            elif o == 'choice_shuffled_order':
                letter = choice_shuffled_order
                histograms_options[target_item_id][letter] = histograms_options[target_item_id].get(letter, 0) + 1
            
            if choice_original_order == correct_original_order:
                    column_name = 'human_accuracy'
                    histograms_correct[target_item_id][column_name] = histograms_correct[target_item_id].get(column_name, 0) + 1
            else:
                    column_name = 'WRONG'
                    histograms_correct[target_item_id][column_name] = histograms_correct[target_item_id].get(column_name, 0) + 1    

    print('record_count', record_count)
    #print('histogram_correct')
    #for key in  sorted(histograms_correct.keys()):
    #    print(f'{key}: {histograms_correct[key]}')

    #print('dict_correct_option')
    #for key in  sorted(dic_correct_option.keys()):
    #    print(f'{key}: {dic_correct_option[key]}')


    #print('histogram_options', histograms_options)
    # Convert the histograms to a DataFrame for better presentation
    histogram_options_df = pd.DataFrame(histograms_options).sort_index()
    histogram_options_df.fillna(0, inplace=True)
    histogram_options_df = histogram_options_df.sort_index(axis=1)

    histogram_correct_df = pd.DataFrame(histograms_correct).sort_index()
    histogram_correct_df.fillna(0, inplace=True)
    histogram_correct_df = histogram_correct_df.sort_index(axis=1)

    # Display the resulting histogram DataFrame
    normalized_histogram_options_df = histogram_options_df/histogram_options_df.sum()
    normalized_histogram_options_df = normalized_histogram_options_df.transpose()
    normalized_histogram_options_df['human_entropy'] = normalized_histogram_options_df.apply(entropy, axis=1) # +  np.random.uniform(-0.001, 0.001, len(normalized_histogram_df['A']))  
    normalized_histogram_options_df['human_item_id'] = range(1, len(normalized_histogram_options_df) + 1)

    normalized_histogram_correct_df = histogram_correct_df/histogram_correct_df.sum()
    normalized_histogram_correct_df = normalized_histogram_correct_df.transpose()
    normalized_histogram_correct_df['human_item_id'] = range(1, len(normalized_histogram_correct_df) + 1)
   
    normalized_histogram_options_df['CORRECT_OPTION_HUMAN'] =  [value for key, value in sorted(dic_correct_option.items())]
    histogram_df = pd.merge(normalized_histogram_options_df, normalized_histogram_correct_df, how='left', on='human_item_id')
    return pd.merge(histogram_df, itens_df, how='left', on='human_item_id')


     

In [3]:
import numpy as np

# 2022
# CH 1062 
# MT 1082
# CN 1092

# 2021
# CH 886
# CN 916
# MT 906

# 2020
# CH 574
# CN 604
# MT 594

# 2019
# CH 520
# MT 522
# CN 519

# 2018
# CH 464
# MT 466

ch_tuples = [(2022, 'CH', 1062), (2021, 'CH', 886), (2020, 'CH', 574), (2019, 'CH', 520), (2018, 'CH', 464)]
cn_tuples = [(2022, 'CN', 1092), (2021, 'CN', 916), (2020, 'CN', 604), (2019, 'CN', 519), (2018, 'CN', 463)]
mt_tuples = [(2022, 'MT', 1082), (2021, 'MT', 906), (2020, 'MT', 594), (2019, 'MT', 522), (2018, 'MT', 466)]
lc_tuples = [(2022, 'LC', 1072), (2021, 'LC', 896), (2020, 'LC', 584), (2019, 'LC', 521)]


for tuple in lc_tuples:
    print('reading enem...', tuple)
    
    year = tuple[0]
    PROVA = tuple[1]
    co_prova = tuple[2]
    
    columns = ['CO_PROVA_'+PROVA,'NU_NOTA_'+PROVA,'TX_RESPOSTAS_'+PROVA,'TX_GABARITO_'+PROVA, 'TP_PRESENCA_'+PROVA]

    enem_human_df = pd.read_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/responses/test_responses_humans_{PROVA}_{year}.csv")
    enem_human_df.sort_values(by='NU_NOTA', inplace=True, ascending=False)

    #enem_human_df = pd.read_csv(f"C:/Users/pedro/Downloads/TRI/microdados/microdados_enem_{year}/DADOS/MICRODADOS_ENEM_{year}.csv", usecols=columns, sep=";",  encoding='latin') 
    item_df_path = f"c:/Users/pedro/Downloads/TRI/microdados/microdados_enem_{year}/DADOS/ITENS_PROVA_{year}.csv"

    print('enem read.')
    percentiles = [0, 0.25, 0.50, 0.75, 1.0]
    str_percentiles = [0, 25, 50, 75, 100]
    #percentiles = [0,1]
    #percentiles = [0, 50, 100]
    nota_percentis = []
    percentil_idx = 0
    for p in percentiles:
        nota = enem_human_df['NU_NOTA'].quantile(p)
        print('nota', nota)
        nota_percentis.append(nota)

    print('grouping...')
    grouped = enem_human_df.groupby('CO_PROVA')['NU_NOTA'].count().sort_values(ascending=False)
    print(grouped)

    co_provas = enem_human_df['CO_PROVA'].unique()
    enem_human_df.sort_values(by='NU_NOTA', inplace=True, ascending=False)

    all_dfs = []
    for id_percentil in range(0, len(nota_percentis) - 1):
         min_nota = nota_percentis[id_percentil]
         max_nota = nota_percentis[id_percentil+1]
         string_pct = str(str_percentiles[id_percentil]) + "_" + str(str_percentiles[id_percentil+1])
            
         print(min_nota, max_nota)   
         df = generate_human_histogram_df(enem_human_df, PROVA=PROVA, CO_PROVA=co_prova, o='choice_original_order', min_nota=min_nota,max_nota=max_nota)
         df.rename(columns={'A': 'human_A_'+string_pct, 'B': 'human_B_'+string_pct, 'C': 'human_C_'+string_pct, 'D': 'human_D_'+string_pct, 'E': 'human_E_'+string_pct}, inplace=True)
         df.rename(columns={'human_accuracy':'human_accuracy_'+string_pct},inplace=True)
         df.rename(columns={'human_entropy':'human_entropy_'+string_pct},inplace=True)
         if 'WRONG' in df.columns:
               df.drop(columns='WRONG', inplace=True)
         all_dfs.append(df)
         filename = f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/itens/intermediarios/human_itens_{PROVA}_{int(co_prova)}_{year}_{string_pct}.csv"
         print("SAVING", filename, df.columns)
         df.to_csv(filename)
        
    result_df = pd.concat(all_dfs, axis=1, verify_integrity=False)
    print('previous columns',sorted(result_df.columns))
    result_df = result_df.loc[:,~result_df.T.duplicated(keep='last')]
    print('final columns', sorted(result_df.columns))
    result_df.to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/itens/human_itens_{PROVA}_{int(co_prova)}_{year}.csv")

        #df = generate_human_histogram_df(enem_human_df, PROVA=exam, CO_PROVA=co_prova, o='correct_percentage', normalize=True)
        #df.to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/2022/correct_percentage/correct_percentage_{exam}_{int(co_prova)}.csv")

print("FINISHED.")     


reading enem... (2022, 'LC', 1072)
enem read.
nota 0.0
nota 447.9
nota 501.9
nota 548.5
nota 776.3
grouping...
CO_PROVA
1068.0    246758
1067.0    242723
1065.0    242370
1066.0    242063
1182.0      3177
1181.0      3065
1179.0      3058
1180.0      3001
1073.0       441
1072.0       223
1146.0        92
1145.0        81
1148.0        78
1147.0        71
Name: NU_NOTA, dtype: int64
0.0 447.9
len_itens_df 1305
record_count 247103
SAVING C:/Users/pedro/Downloads/TRI/test_responses_humans/2022/itens/intermediarios/human_itens_LC_1072_2022_0_25.csv Index(['human_A_0_25', 'human_B_0_25', 'human_C_0_25', 'human_D_0_25',
       'human_E_0_25', 'human_entropy_0_25', 'human_item_id',
       'CORRECT_OPTION_HUMAN', 'human_accuracy_0_25', 'CO_POSICAO', 'SG_AREA',
       'CO_ITEM', 'TX_GABARITO', 'CO_HABILIDADE', 'IN_ITEM_ABAN',
       'TX_MOTIVO_ABAN', 'NU_PARAM_A', 'NU_PARAM_B', 'NU_PARAM_C', 'TX_COR',
       'CO_PROVA', 'TP_LINGUA', 'IN_ITEM_ADAPTADO'],
      dtype='object')
447.9 501.9
len_it